In [1]:
import duckdb as ddb
import pandas as pd
import geopandas as gpd
from ddb_spatial_tools import *
import folium

ddb.load_extension('spatial')


In [2]:
optionA = ddb.sql('SELECT time_of_trip,start_lat,start_lng FROM "NYC/other-Lyft_B02510.csv"').df()

In [4]:
type(optionA)

pandas.core.frame.DataFrame

In [5]:
#ddb.sql('CREATE TABLE nyc AS SELECT time_of_trip,start_lat,start_lng FROM "NYC/other-Lyft_B02510.csv"')

In [6]:
# Drop the nyc table if it exists
#ddb.sql('DROP TABLE IF EXISTS nyc')

In [3]:
optionA

,time_of_trip,start_lat,start_lng
0,9/4/2014 9:51,40.64705,-73.77988
1,8/27/2014 21:13,40.74916,-73.98373
2,9/4/2014 14:16,40.64065,-73.97594
3,9/4/2014 16:08,40.75002,-73.99514
4,8/28/2014 2:41,40.76715,-73.98636
...,...,...,...
267696,9/7/2014 3:19,40.71931,-73.99138
267697,9/7/2014 2:59,40.73868,-73.98862
267698,9/7/2014 1:55,40.71554,-73.94877
267699,9/5/2014 23:26,40.74667,-73.97445


In [7]:
ddb.sql("SELECT install_path FROM duckdb_extensions() WHERE extension_name='spatial'")

┌──────────────────────────────────────────────────────────────────────────┐
│                               install_path                               │
│                                 varchar                                  │
├──────────────────────────────────────────────────────────────────────────┤
│ /Users/jeff/.duckdb/extensions/v0.8.1/osx_amd64/spatial.duckdb_extension │
└──────────────────────────────────────────────────────────────────────────┘

In [5]:
ddb.sql("SELECT *, ST_Point(start_lng, start_lat) as start_point FROM optionA")

┌─────────────────┬───────────┬───────────┬────────────────────────────┐
│  time_of_trip   │ start_lat │ start_lng │        start_point         │
│     varchar     │  double   │  double   │          geometry          │
├─────────────────┼───────────┼───────────┼────────────────────────────┤
│ 9/4/2014 9:51   │  40.64705 │ -73.77988 │ POINT (-73.77988 40.64705) │
│ 8/27/2014 21:13 │  40.74916 │ -73.98373 │ POINT (-73.98373 40.74916) │
│ 9/4/2014 14:16  │  40.64065 │ -73.97594 │ POINT (-73.97594 40.64065) │
│ 9/4/2014 16:08  │  40.75002 │ -73.99514 │ POINT (-73.99514 40.75002) │
│ 8/28/2014 2:41  │  40.76715 │ -73.98636 │ POINT (-73.98636 40.76715) │
│ 9/13/2014 3:57  │  40.70707 │ -74.01211 │ POINT (-74.01211 40.70707) │
│ 9/11/2014 23:13 │  40.72953 │ -73.97799 │ POINT (-73.97799 40.72953) │
│ 8/27/2014 22:56 │  40.76689 │ -73.95631 │ POINT (-73.95631 40.76689) │
│ 9/10/2014 17:48 │  40.82332 │ -73.86235 │ POINT (-73.86235 40.82332) │
│ 9/12/2014 23:24 │  40.72188 │ -73.99732 │ POINT (

In [8]:
gdf = gpd.GeoDataFrame(optionA, geometry=gpd.points_from_xy(optionA.start_lng, optionA.start_lat))

In [9]:
gdf

,time_of_trip,start_lat,start_lng,geometry
0,9/4/2014 9:51,40.64705,-73.77988,POINT (-73.77988 40.64705)
1,8/27/2014 21:13,40.74916,-73.98373,POINT (-73.98373 40.74916)
2,9/4/2014 14:16,40.64065,-73.97594,POINT (-73.97594 40.64065)
3,9/4/2014 16:08,40.75002,-73.99514,POINT (-73.99514 40.75002)
4,8/28/2014 2:41,40.76715,-73.98636,POINT (-73.98636 40.76715)
...,...,...,...,...
267696,9/7/2014 3:19,40.71931,-73.99138,POINT (-73.99138 40.71931)
267697,9/7/2014 2:59,40.73868,-73.98862,POINT (-73.98862 40.73868)
267698,9/7/2014 1:55,40.71554,-73.94877,POINT (-73.94877 40.71554)
267699,9/5/2014 23:26,40.74667,-73.97445,POINT (-73.97445 40.74667)


In [7]:
# Export the GeoDataFrame to parquet
gdf.to_parquet('nyc.parquet')

In [10]:
parquet = ddb.read_parquet('nyc.parquet')

In [11]:
parquet

┌─────────────────┬───────────┬───────────┬────────────────────────────────────────────────────────────────────┐
│  time_of_trip   │ start_lat │ start_lng │                              geometry                              │
│     varchar     │  double   │  double   │                                blob                                │
├─────────────────┼───────────┼───────────┼────────────────────────────────────────────────────────────────────┤
│ 9/4/2014 9:51   │  40.64705 │ -73.77988 │ \x01\x01\x00\x00\x00}\xB3\xCD\x8D\xE9qR\xC0;p\xCE\x88\xD2RD@       │
│ 8/27/2014 21:13 │  40.74916 │ -73.98373 │ \x01\x01\x00\x00\x00\x05\x86\xACn\xF5~R\xC0V\xBC\x91y\xE4_D@       │
│ 9/4/2014 14:16  │  40.64065 │ -73.97594 │ \x01\x01\x00\x00\x00\xED\xB6\x0B\xCDu~R\xC0Y\x17\xB7\xD1\x00RD@    │
│ 9/4/2014 16:08  │  40.75002 │ -73.99514 │ \x01\x01\x00\x00\x00A\xBC\xAE_\xB0\x7FR\xC0G\xAC\xC5\xA7\x00`D@    │
│ 8/28/2014 2:41  │  40.76715 │ -73.98636 │ \x01\x01\x00\x00\x00I\x85\xB1\x85 \x7FR\xC0.\x90\xA0

In [26]:
ddb.sql("DROP TABLE IF EXISTS nyc_geoparquet")

In [12]:
ddb.sql(f"CREATE TABLE nyc_geoparquet AS SELECT * EXCLUDE geometry, ST_GEOMFROMWKB(geometry) AS geometry FROM 'parquet'")
# CREATE TABLE attapeu AS SELECT * EXCLUDE geometry, ST_GEOMFROMWKB(geometry) AS geometry FROM 
#'s3://us-west-2.opendata.source.coop/google-research-open-buildings/geoparquet-admin1/country=LAO/Attapeu.parquet';

In [13]:
ddb.sql('SELECT * FROM nyc_geoparquet WHERE ST_Y(geometry) > 40')

┌─────────────────┬───────────┬───────────┬────────────────────────────┐
│  time_of_trip   │ start_lat │ start_lng │          geometry          │
│     varchar     │  double   │  double   │          geometry          │
├─────────────────┼───────────┼───────────┼────────────────────────────┤
│ 9/4/2014 9:51   │  40.64705 │ -73.77988 │ POINT (-73.77988 40.64705) │
│ 8/27/2014 21:13 │  40.74916 │ -73.98373 │ POINT (-73.98373 40.74916) │
│ 9/4/2014 14:16  │  40.64065 │ -73.97594 │ POINT (-73.97594 40.64065) │
│ 9/4/2014 16:08  │  40.75002 │ -73.99514 │ POINT (-73.99514 40.75002) │
│ 8/28/2014 2:41  │  40.76715 │ -73.98636 │ POINT (-73.98636 40.76715) │
│ 9/13/2014 3:57  │  40.70707 │ -74.01211 │ POINT (-74.01211 40.70707) │
│ 9/11/2014 23:13 │  40.72953 │ -73.97799 │ POINT (-73.97799 40.72953) │
│ 8/27/2014 22:56 │  40.76689 │ -73.95631 │ POINT (-73.95631 40.76689) │
│ 9/10/2014 17:48 │  40.82332 │ -73.86235 │ POINT (-73.86235 40.82332) │
│ 9/12/2014 23:24 │  40.72188 │ -73.99732 │ POINT (

In [44]:
# Get the max latitude, min latitude, max longitude, and min longitude
max_lat = ddb.sql('SELECT MAX(ST_Y(geometry)) FROM nyc_geoparquet').df()
min_lat = ddb.sql('SELECT MIN(ST_Y(geometry)) FROM nyc_geoparquet').df()
max_lon = ddb.sql('SELECT MAX(ST_X(geometry)) FROM nyc_geoparquet').df()
min_lon = ddb.sql('SELECT MIN(ST_X(geometry)) FROM nyc_geoparquet').df()


In [78]:
min_lat

,min(st_y(geometry))
0,40.52785


In [79]:
# Add 0.1 to the min latitude
min_lat = min_lat + 0.1
min_lat

,min(st_y(geometry))
0,40.62785


In [75]:
max_lat

,max(st_y(geometry))
0,40.93562


In [80]:
# Subtract 0.1 from the max latitude
max_lat = max_lat - 0.1
max_lat

,max(st_y(geometry))
0,40.83562


In [91]:

# Use ST_GeomFromText to create a bounding box polygon
#bbox = ddb.sql(f"SELECT ST_GeomFromText('POLYGON(({min_lon.iloc[0,0]} {min_lat.iloc[0,0]}, {min_lon.iloc[0,0]} {max_lat.iloc[0,0]}, {max_lon.iloc[0,0]} {max_lat.iloc[0,0]}, {max_lon.iloc[0,0]} {min_lat.iloc[0,0]}, {min_lon.iloc[0,0]} {min_lat.iloc[0,0]})))')")
bbox = ddb.sql(f"SELECT ST_GeomFromText('POLYGON(({min_lon.iloc[0,0]} {min_lat.iloc[0,0]}, {min_lon.iloc[0,0]} {max_lat.iloc[0,0]}, {max_lon.iloc[0,0]} {max_lat.iloc[0,0]}, {max_lon.iloc[0,0]} {min_lat.iloc[0,0]}, {min_lon.iloc[0,0]} {min_lat.iloc[0,0]})))') AS geometry")

In [97]:
# Create table AOI and add bbox
ddb.sql('DROP TABLE IF EXISTS AOI')

ddb.sql('CREATE TABLE AOI AS SELECT * FROM bbox')
ddb.table('AOI')





┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                    geometry                                                    │
│                                                    geometry                                                    │
├────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((-74.22689 40.62785, -74.22689 40.83562, -73.72315 40.83562, -73.72315 40.62785, -74.22689 40.62785)) │
└────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [99]:
# Select all points that are within the bounding box
ddb.sql('SELECT * FROM nyc_geoparquet WHERE ST_Contains((SELECT geometry FROM AOI), geometry)').df()

,time_of_trip,start_lat,start_lng,geometry
0,9/4/2014 9:51,40.64705,-73.77988,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
1,8/27/2014 21:13,40.74916,-73.98373,"[0, 0, 24, 0, 58, 52, 51, 56, 0, 0, 0, 0, 1, 0..."
2,9/4/2014 14:16,40.64065,-73.97594,"[0, 0, 24, 0, 47, 49, 50, 47, 0, 0, 0, 0, 1, 0..."
3,9/4/2014 16:08,40.75002,-73.99514,"[0, 0, 24, 0, 56, 47, 50, 48, 0, 0, 0, 0, 1, 0..."
4,8/28/2014 2:41,40.76715,-73.98636,"[0, 0, 24, 0, 48, 49, 52, 32, 0, 0, 0, 0, 1, 0..."
...,...,...,...,...
258107,9/7/2014 3:19,40.71931,-73.99138,"[0, 0, 24, 0, 116, 147, 24, 126, 0, 0, 0, 0, 1..."
258108,9/7/2014 2:59,40.73868,-73.98862,"[0, 0, 24, 0, 94, 68, 64, 1, 0, 0, 0, 0, 1, 0,..."
258109,9/7/2014 1:55,40.71554,-73.94877,"[0, 0, 24, 0, 72, 162, 127, 82, 0, 0, 0, 0, 1,..."
258110,9/5/2014 23:26,40.74667,-73.97445,"[0, 0, 24, 0, 68, 64, 1, 1, 0, 0, 0, 0, 1, 0, ..."


In [20]:
# ST_Transform the geometry to Canada Albers Equal Area Conic
ddb.sql('SELECT ST_Transform(geometry, 102001) FROM nyc_geoparquet')

BinderException: Binder Error: No function matches the given name and argument types 'st_transform(GEOMETRY, INTEGER)'. You might need to add explicit type casts.
	Candidate functions:
	st_transform(BOX_2D, VARCHAR, VARCHAR) -> BOX_2D
	st_transform(POINT_2D, VARCHAR, VARCHAR) -> POINT_2D
	st_transform(GEOMETRY, VARCHAR, VARCHAR) -> GEOMETRY


In [81]:
ddb.sql("DROP TABLE IF EXISTS points_db")

In [82]:
ddb.sql("""
        CREATE TABLE points_db AS
        SELECT time_of_trip, ST_GEOMFROMWKB(geometry) GEO, ST_Transform((ST_FlipCoordinates(GEO)), 'epsg:4326', 'esri:102001') AS AEAC
        FROM 'nyc.parquet'
        """)

In [83]:
ddb.sql("""
        SELECT *
        FROM points_db
        """)

┌─────────────────┬────────────────────────────┬───────────────────────────────────────────────┐
│  time_of_trip   │            GEO             │                     AEAC                      │
│     varchar     │          geometry          │                   geometry                    │
├─────────────────┼────────────────────────────┼───────────────────────────────────────────────┤
│ 9/4/2014 9:51   │ POINT (-73.77988 40.64705) │ POINT (1897449.0255942738 386504.2102560082)  │
│ 8/27/2014 21:13 │ POINT (-73.98373 40.74916) │ POINT (1877121.894533895 391214.34149144695)  │
│ 9/4/2014 14:16  │ POINT (-73.97594 40.64065) │ POINT (1881536.4379089037 380335.4551824107)  │
│ 9/4/2014 16:08  │ POINT (-73.99514 40.75002) │ POINT (1876154.2327036646 390983.55843685986) │
│ 8/28/2014 2:41  │ POINT (-73.98636 40.76715) │ POINT (1876280.1320683847 392981.10363323)    │
│ 9/13/2014 3:57  │ POINT (-74.01211 40.70707) │ POINT (1876251.1827902275 386116.20897549985) │
│ 9/11/2014 23:13 │ POINT (-73

In [84]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np


xmin = ddb.sql('SELECT MIN(ST_X(AEAC)) FROM points_db').df().iloc[0,0]
ymin = ddb.sql('SELECT MIN(ST_Y(AEAC)) FROM points_db').df().iloc[0,0]
xmax = ddb.sql('SELECT MAX(ST_X(AEAC)) FROM points_db').df().iloc[0,0]
ymax = ddb.sql('SELECT MAX(ST_Y(AEAC)) FROM points_db').df().iloc[0,0]

length = 3000
wide = 3000

cols = list(np.arange(xmin, xmax + wide, wide))
rows = list(np.arange(ymin, ymax + length, length))

polygons = []
for x in cols[:-1]:
    for y in rows[:-1]:
        polygons.append(Polygon([(x,y), (x+wide, y), (x+wide, y+length), (x, y+length)]))

grid = gpd.GeoDataFrame({'geometry':polygons})

In [85]:
grid

,geometry
0,"POLYGON ((1864597.207 361800.031, 1867597.207 ..."
1,"POLYGON ((1864597.207 364800.031, 1867597.207 ..."
2,"POLYGON ((1864597.207 367800.031, 1867597.207 ..."
3,"POLYGON ((1864597.207 370800.031, 1867597.207 ..."
4,"POLYGON ((1864597.207 373800.031, 1867597.207 ..."
...,...
229,"POLYGON ((1900597.207 400800.031, 1903597.207 ..."
230,"POLYGON ((1900597.207 403800.031, 1903597.207 ..."
231,"POLYGON ((1900597.207 406800.031, 1903597.207 ..."
232,"POLYGON ((1900597.207 409800.031, 1903597.207 ..."


In [86]:
grid.set_crs(crs='esri:102001', inplace=True)

,geometry
0,"POLYGON ((1864597.207 361800.031, 1867597.207 ..."
1,"POLYGON ((1864597.207 364800.031, 1867597.207 ..."
2,"POLYGON ((1864597.207 367800.031, 1867597.207 ..."
3,"POLYGON ((1864597.207 370800.031, 1867597.207 ..."
4,"POLYGON ((1864597.207 373800.031, 1867597.207 ..."
...,...
229,"POLYGON ((1900597.207 400800.031, 1903597.207 ..."
230,"POLYGON ((1900597.207 403800.031, 1903597.207 ..."
231,"POLYGON ((1900597.207 406800.031, 1903597.207 ..."
232,"POLYGON ((1900597.207 409800.031, 1903597.207 ..."


In [88]:
m

In [49]:
ddb.sql("CREATE TABLE grid AS SELECT * EXCLUDE geometry, ST_GEOMFROMWKB(geometry) GEO FROM grid")

NotImplementedException: Not implemented Error: Data type 'geometry' not recognized

# Using ddb_spatial_tools

In [2]:
if read_points_from_parquet_file('nyc.parquet', 'nyc'):
    ddb.table('nyc').show()

INFO:root:Created table 'nyc' from nyc.parquet


┌─────────────────┬───────────┬───────────┬──────────────────────┬──────────────────────┬──────────────────────────────┐
│  time_of_trip   │ start_lat │ start_lng │       geometry       │         GEO          │             AEAC             │
│     varchar     │  double   │  double   │         blob         │       geometry       │           geometry           │
├─────────────────┼───────────┼───────────┼──────────────────────┼──────────────────────┼──────────────────────────────┤
│ 9/4/2014 9:51   │  40.64705 │ -73.77988 │ \x01\x01\x00\x00\x…  │ POINT (-73.77988 4…  │ POINT (1897449.0255942738 …  │
│ 8/27/2014 21:13 │  40.74916 │ -73.98373 │ \x01\x01\x00\x00\x…  │ POINT (-73.98373 4…  │ POINT (1877121.894533895 3…  │
│ 9/4/2014 14:16  │  40.64065 │ -73.97594 │ \x01\x01\x00\x00\x…  │ POINT (-73.97594 4…  │ POINT (1881536.4379089037 …  │
│ 9/4/2014 16:08  │  40.75002 │ -73.99514 │ \x01\x01\x00\x00\x…  │ POINT (-73.99514 4…  │ POINT (1876154.2327036646 …  │
│ 8/28/2014 2:41  │  40.76715 │ 

In [3]:
grid_size = 5000 # in meters; the length and width of each grid cell
crs_code = 'esri:102001' # the CRS to use for the output grid. Example: 'esri:102001' for Canada Albers Equal Area Conic; 'epsg:4326' for Geo/WGS84
grid = create_vector_grid('nyc', 'AEAC', grid_size, crs_code)

In [4]:
points = ddb.sql('SELECT * FROM nyc')

In [5]:
points

┌─────────────────┬───────────┬───────────┬──────────────────────┬──────────────────────┬──────────────────────────────┐
│  time_of_trip   │ start_lat │ start_lng │       geometry       │         GEO          │             AEAC             │
│     varchar     │  double   │  double   │         blob         │       geometry       │           geometry           │
├─────────────────┼───────────┼───────────┼──────────────────────┼──────────────────────┼──────────────────────────────┤
│ 9/4/2014 9:51   │  40.64705 │ -73.77988 │ \x01\x01\x00\x00\x…  │ POINT (-73.77988 4…  │ POINT (1897449.0255942738 …  │
│ 8/27/2014 21:13 │  40.74916 │ -73.98373 │ \x01\x01\x00\x00\x…  │ POINT (-73.98373 4…  │ POINT (1877121.894533895 3…  │
│ 9/4/2014 14:16  │  40.64065 │ -73.97594 │ \x01\x01\x00\x00\x…  │ POINT (-73.97594 4…  │ POINT (1881536.4379089037 …  │
│ 9/4/2014 16:08  │  40.75002 │ -73.99514 │ \x01\x01\x00\x00\x…  │ POINT (-73.99514 4…  │ POINT (1876154.2327036646 …  │
│ 8/28/2014 2:41  │  40.76715 │ 

In [6]:
from shapely import wkt

# Convert the geometry column to WKT
points = ddb.sql("SELECT time_of_trip, ST_AsText(AEAC) AS WKT FROM nyc LIMIT 10000").df()
points_gdf = gpd.GeoDataFrame(points, geometry=points.WKT.apply(wkt.loads), crs='esri:102001')

In [7]:
# Create a folium map
import folium

m = folium.Map(location=[45.5, -73.5], zoom_start=10)

# Add the grid to the map
folium.GeoJson(grid, name=f'Grid ({grid_size}x{grid_size}m)').add_to(m)
#folium.GeoJson(points_gdf).add_to(m)
folium.LayerControl().add_to(m)





DEBUG:pyproj:PROJ_ERROR: hgridshift: could not find required grid(s).
DEBUG:pyproj:PROJ_ERROR: pipeline: Pipeline: Bad step definition: proj=hgridshift (File not found or invalid)
DEBUG:pyproj:PROJ_ERROR: hgridshift: could not find required grid(s).
DEBUG:pyproj:PROJ_ERROR: pipeline: Pipeline: Bad step definition: proj=hgridshift (File not found or invalid)
DEBUG:pyproj:PROJ_ERROR: hgridshift: could not find required grid(s).
DEBUG:pyproj:PROJ_ERROR: pipeline: Pipeline: Bad step definition: proj=hgridshift (File not found or invalid)
DEBUG:pyproj:PROJ_ERROR: hgridshift: could not find required grid(s).
DEBUG:pyproj:PROJ_ERROR: pipeline: Pipeline: Bad step definition: proj=hgridshift (File not found or invalid)
DEBUG:pyproj:PROJ_ERROR: hgridshift: could not find required grid(s).
DEBUG:pyproj:PROJ_ERROR: pipeline: Pipeline: Bad step definition: proj=hgridshift (File not found or invalid)
DEBUG:pyproj:PROJ_ERROR: hgridshift: could not find required grid(s).
DEBUG:pyproj:PROJ_ERROR: pipel

In [8]:
m.fit_bounds(m.get_bounds())

m

In [9]:
# Use shapely to create a lat column and a lon column
points_gdf = points_gdf.to_crs('epsg:4326')
points_gdf['lat'] = points_gdf.geometry.y
points_gdf['lon'] = points_gdf.geometry.x
points_gdf

,time_of_trip,WKT,geometry,lat,lon
0,9/4/2014 9:51,POINT (1897449.0255942738 386504.2102560082),POINT (-73.77988 40.64705),40.64705,-73.77988
1,8/27/2014 21:13,POINT (1877121.894533895 391214.34149144695),POINT (-73.98373 40.74916),40.74916,-73.98373
2,9/4/2014 14:16,POINT (1881536.4379089037 380335.4551824107),POINT (-73.97594 40.64065),40.64065,-73.97594
3,9/4/2014 16:08,POINT (1876154.2327036646 390983.55843685986),POINT (-73.99514 40.75002),40.75002,-73.99514
4,8/28/2014 2:41,POINT (1876280.1320683847 392981.10363323),POINT (-73.98636 40.76715),40.76715,-73.98636
...,...,...,...,...,...
9995,8/15/2014 21:32,POINT (1875738.7675416435 391634.2910436794),POINT (-73.99726 40.75696),40.75696,-73.99726
9996,9/19/2014 23:41,POINT (1876570.8236650326 388513.34550204116),POINT (-73.99975 40.72713),40.72713,-73.99975
9997,8/15/2014 5:26,POINT (1877178.4624578927 390926.6127267758),POINT (-73.98418 40.74647),40.74647,-73.98418
9998,9/3/2014 2:10,POINT (1885179.1561083663 397094.6191699316),POINT (-73.87399 40.77661),40.77661,-73.87399


In [18]:
# Create a heatmap from the points
from folium import plugins

m = folium.Map(location=[45.5, -73.5], zoom_start=10, tiles='cartodbdark_matter') #cartodbpositron, cartodbdark_matter

# Change crs to epsg:4326
points_geo = points_gdf.to_crs('epsg:4326')

# Create a heatmap from the points


folium.GeoJson(grid, name=f'Grid ({grid_size}x{grid_size}m)').add_to(m)
m.add_child(plugins.HeatMap(data=points_geo[['lat', 'lon']].values.tolist(), radius=3, blur=3, name='Heatmap'))
m.fit_bounds(m.get_bounds())
# Add layer control
folium.LayerControl().add_to(m)


In [19]:
m